# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [3]:
import pandas as pd

# Carregar a base de dados
df = pd.read_csv("base_final.csv")

# Renomear a coluna 'default' para 'resposta'
df.rename(columns={'default': 'resposta'}, inplace=True)

# Criar um mapeamento para a variável resposta
mapping = {False: 'bom', True: 'mau'}

# Aplicar o mapeamento
df['resposta'] = df['resposta'].map(mapping)

# Exibir tipos de dados das variáveis
print("Tipos de dados:\n", df.dtypes)

# Contar valores ausentes
print("\nValores ausentes:\n", df.isnull().sum())

# Distribuição da variável resposta
print("\nDistribuição da variável resposta:\n", df['resposta'].value_counts(normalize=True) * 100)

Tipos de dados:
 ID                       int64
CODE_GENDER             object
FLAG_OWN_CAR            object
FLAG_OWN_REALTY         object
CNT_CHILDREN             int64
AMT_INCOME_TOTAL       float64
NAME_INCOME_TYPE        object
NAME_EDUCATION_TYPE     object
NAME_FAMILY_STATUS      object
NAME_HOUSING_TYPE       object
DAYS_BIRTH               int64
DAYS_EMPLOYED            int64
FLAG_MOBIL               int64
FLAG_WORK_PHONE          int64
FLAG_PHONE               int64
FLAG_EMAIL               int64
OCCUPATION_TYPE         object
CNT_FAM_MEMBERS        float64
resposta                object
dtype: object

Valores ausentes:
 ID                        0
CODE_GENDER               0
FLAG_OWN_CAR              0
FLAG_OWN_REALTY           0
CNT_CHILDREN              0
AMT_INCOME_TOTAL          0
NAME_INCOME_TYPE          0
NAME_EDUCATION_TYPE       0
NAME_FAMILY_STATUS        0
NAME_HOUSING_TYPE         0
DAYS_BIRTH                0
DAYS_EMPLOYED             0
FLAG_MOBIL              

#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável.
    Dica:
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis,
        3. conte o número de categorias dessa variável
        4. acumule essa informação de 3. na lista que você criou em 1.
        5. No final, essa lista pode ser atribuída à nossa variável.

In [4]:
# Criar um DataFrame com os nomes das variáveis e seus tipos
metadados = pd.DataFrame({
    "variavel": df.columns,
    "tipo_dado": df.dtypes.values
})

# Criar uma lista para armazenar a quantidade de categorias
qtd_categorias = []

# Contar o número de categorias para cada variável
for coluna in df.columns:
    qtd_categorias.append(df[coluna].nunique())

# Adicionar a lista ao DataFrame de metadados
metadados["qtd_categorias"] = qtd_categorias

# Exibir o DataFrame de metadados
print(metadados)

               variavel tipo_dado  qtd_categorias
0                    ID     int64           16650
1           CODE_GENDER    object               2
2          FLAG_OWN_CAR    object               2
3       FLAG_OWN_REALTY    object               2
4          CNT_CHILDREN     int64               8
5      AMT_INCOME_TOTAL   float64             221
6      NAME_INCOME_TYPE    object               5
7   NAME_EDUCATION_TYPE    object               5
8    NAME_FAMILY_STATUS    object               5
9     NAME_HOUSING_TYPE    object               6
10           DAYS_BIRTH     int64            5298
11        DAYS_EMPLOYED     int64            3005
12           FLAG_MOBIL     int64               1
13      FLAG_WORK_PHONE     int64               2
14           FLAG_PHONE     int64               2
15           FLAG_EMAIL     int64               2
16      OCCUPATION_TYPE    object              18
17      CNT_FAM_MEMBERS   float64               9
18             resposta    object               2


#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [5]:
# Selecionar apenas variáveis categóricas (tipo object) para dummificação
variaveis_categoricas = df.select_dtypes(include=['object']).columns

# Remover a variável 'resposta' (pois é nossa variável-alvo)
variaveis_categoricas = variaveis_categoricas.drop('resposta')

# Criar variáveis dummy
df_dummies = pd.get_dummies(df, columns=variaveis_categoricas, drop_first=True)

# Remover colunas originais de texto (que foram convertidas em dummies)
df_final = df_dummies.select_dtypes(exclude=['object'])

# Exibir as primeiras linhas do dataframe final
print(df_final.head())

        ID  CNT_CHILDREN  AMT_INCOME_TOTAL  DAYS_BIRTH  DAYS_EMPLOYED  \
0  5008806             0          112500.0      -21474          -1134   
1  5008810             0          270000.0      -19110          -3051   
2  5008811             0          270000.0      -19110          -3051   
3  5112956             0          270000.0      -16872           -769   
4  5008825             0          130500.0      -10669          -1103   

   FLAG_MOBIL  FLAG_WORK_PHONE  FLAG_PHONE  FLAG_EMAIL  CNT_FAM_MEMBERS  ...  \
0           1                0           0           0              2.0  ...   
1           1                0           1           1              1.0  ...   
2           1                0           1           1              1.0  ...   
3           1                1           1           1              2.0  ...   
4           1                0           0           0              2.0  ...   

   OCCUPATION_TYPE_Laborers  OCCUPATION_TYPE_Low-skill Laborers  \
0            

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [6]:
# Criar tabela cruzada para FLAG_EMAIL x resposta
tabela_email = pd.crosstab(df['FLAG_EMAIL'], df['resposta'], normalize='index') * 100

# Criar tabela cruzada para FLAG_OWN_CAR x resposta
tabela_carro = pd.crosstab(df['FLAG_OWN_CAR'], df['resposta'], normalize='index') * 100

# Exibir os resultados
print("Tabela cruzada - FLAG_EMAIL x resposta:")
print(tabela_email)
print("\nTabela cruzada - FLAG_OWN_CAR x resposta:")
print(tabela_carro)

Tabela cruzada - FLAG_EMAIL x resposta:
resposta          bom       mau
FLAG_EMAIL                     
0           97.626895  2.373105
1           97.972973  2.027027

Tabela cruzada - FLAG_OWN_CAR x resposta:
resposta            bom       mau
FLAG_OWN_CAR                     
N             97.573197  2.426803
Y             97.790482  2.209518


#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [7]:
# Salvar o dataframe final
df_final.to_csv("base_tratada.csv", index=False)

print("Base salva como 'base_tratada.csv'.")

Base salva como 'base_tratada.csv'.
